# LEET CODE
## Problem-1321: Restaurant Growth
Class: Medium

Source: https://leetcode.com/problems/restaurant-growth

## Description
Table: Customer
| Column Name   | Type    |
|---------------|---------|
| customer_id   | int     |
| name          | varchar |
| visited_on    | date    |
| amount        | int     |

- In SQL,(customer_id, visited_on) is the primary key for this table.
- This table contains data about customer transactions in a restaurant.
- visited_on is the date on which the customer with ID (customer_id) has visited the restaurant.
- amount is the total paid by a customer.

## To Do
You are the restaurant owner and you want to analyze a possible expansion (there will be at least one customer every day).

Compute the moving average of how much the customer paid in a seven days window (i.e., current day | 6 days before). average_amount should be rounded to two decimal places.

Return the result table ordered by visited_on in ascending order.

The result format is in the following example.

## Example

Input: 

Customer table:
| customer_id | name         | visited_on   | amount      |
|-------------|--------------|--------------|-------------|
| 1           | Jhon         | 2019-01-01   | 100         |
| 2           | Daniel       | 2019-01-02   | 110         |
| 3           | Jade         | 2019-01-03   | 120         |
| 4           | Khaled       | 2019-01-04   | 130         |
| 5           | Winston      | 2019-01-05   | 110         | 
| 6           | Elvis        | 2019-01-06   | 140         | 
| 7           | Anna         | 2019-01-07   | 150         |
| 8           | Maria        | 2019-01-08   | 80          |
| 9           | Jaze         | 2019-01-09   | 110         | 
| 1           | Jhon         | 2019-01-10   | 130         | 
| 3           | Jade         | 2019-01-10   | 150         | 

Output: 
| visited_on   | amount       | average_amount |
|--------------|--------------|----------------|
| 2019-01-07   | 860          | 122.86         |
| 2019-01-08   | 840          | 120            |
| 2019-01-09   | 840          | 120            |
| 2019-01-10   | 1000         | 142.86         |

Explanation: 
- 1st moving average from 2019-01-01 to 2019-01-07 has an average_amount of (100 + 110 + 120 + 130 + 110 + 140 + 150)/7 = 122.86
- 2nd moving average from 2019-01-02 to 2019-01-08 has an average_amount of (110 + 120 + 130 + 110 + 140 + 150 + 80)/7 = 120
- 3rd moving average from 2019-01-03 to 2019-01-09 has an average_amount of (120 + 130 + 110 + 140 + 150 + 80 + 110)/7 = 120
- 4th moving average from 2019-01-04 to 2019-01-10 has an average_amount of (130 + 110 + 140 + 150 + 80 + 110 + 130 + 150)/7 = 142.86


## Importing

In [1]:
import pandas as pd
from pandasql import sqldf

## Define Schema

In [2]:
data = {
    'customer_id': [1, 2, 3, 4, 5, 6, 7, 8, 9, 1, 3],
    'name': ['Jhon', 'Daniel', 'Jade', 'Khaled', 'Winston', 'Elvis', 'Anna', 'Maria', 'Jaze', 'Jhon', 'Jade'],
    'visited_on': ['2019-01-01', '2019-01-02', '2019-01-03', '2019-01-04', '2019-01-05', '2019-01-06', '2019-01-07', '2019-01-08', '2019-01-09', '2019-01-10', '2019-01-10'],
    'amount': [100, 110, 120, 130, 110, 140, 150, 80, 110, 130, 150]
}

Customer = pd.DataFrame(data)

Customer

,customer_id,name,visited_on,amount
0,1,Jhon,2019-01-01,100
1,2,Daniel,2019-01-02,110
2,3,Jade,2019-01-03,120
3,4,Khaled,2019-01-04,130
4,5,Winston,2019-01-05,110
5,6,Elvis,2019-01-06,140
6,7,Anna,2019-01-07,150
7,8,Maria,2019-01-08,80
8,9,Jaze,2019-01-09,110
9,1,Jhon,2019-01-10,130


In [24]:
data = {
    "customer_id": [1, 4, 5, 6, 7, 8, 9, 1, 3],
    "name": ["Jhon", "Khaled", "Winston", "Elvis", "Anna", "Maria", "Jaze", "Jhon", "Jade"],
    "visited_on": ["2019-01-01", "2019-01-04", "2019-01-05", "2019-01-06", "2019-01-07", "2019-01-08", "2019-01-09", "2019-01-10", "2019-01-10"],
    "amount": [100, 130, 110, 140, 150, 80, 110, 130, 150]
}

Customer_2 = pd.DataFrame(data)

Customer_2

,customer_id,name,visited_on,amount
0,1,Jhon,2019-01-01,100
1,4,Khaled,2019-01-04,130
2,5,Winston,2019-01-05,110
3,6,Elvis,2019-01-06,140
4,7,Anna,2019-01-07,150
5,8,Maria,2019-01-08,80
6,9,Jaze,2019-01-09,110
7,1,Jhon,2019-01-10,130
8,3,Jade,2019-01-10,150


## Task

In [57]:
# Define the SQL query
query = """
WITH growth AS (
    SELECT 
        DISTINCT visited_on,
        SUM(amount) OVER (ORDER BY visited_on RANGE BETWEEN INTERVAL 6 DAY PRECEDING AND CURRENT ROW) AS amount,
        MIN(visited_on) OVER() AS 1st_date
    FROM Customer
)
SELECT visited_on, amount, ROUND(amount/7.0, 2) AS average_amount
FROM growth
WHERE visited_on>= 1st_date + 6
;
"""

# Excute the query using pandasql
result = sqldf(query, env={'Customer':Customer})

# Display the result dataframe
display(result)

Expected: 
| visited_on | amount | average_amount |
| ---------- | ------ | -------------- |
| 2019-01-07 | 630    | 90             |
| 2019-01-08 | 610    | 87.14          |
| 2019-01-09 | 720    | 102.86         |
| 2019-01-10 | 1000   | 142.86         |

It will be harder to do it in sqlite

In [45]:
## Solution submitted:
"""
WITH growth AS (
    SELECT 
        DISTINCT visited_on,
        SUM(amount) OVER (ORDER BY visited_on RANGE BETWEEN INTERVAL 6 DAY PRECEDING AND CURRENT ROW) AS amount,
        MIN(visited_on) OVER() AS 1st_date
    FROM Customer
)
SELECT visited_on, amount, ROUND(amount/7.0, 2) AS average_amount
FROM growth
WHERE visited_on>= 1st_date + 6
;
"""

'\nWITH growth AS (\n    SELECT \n        DISTINCT visited_on,\n        SUM(amount) OVER (ORDER BY visited_on RANGE BETWEEN INTERVAL 6 DAY PRECEDING AND CURRENT ROW) AS amount,\n        MIN(visited_on) OVER() AS 1st_date\n    FROM Customer\n)\nSELECT visited_on, amount, ROUND(amount/7.0, 2) AS average_amount\nFROM growth\nWHERE visited_on>= 1st_date + 6\n;\n'

In [43]:
## Old solution submitted:
"""
TH visited_amount AS (
    SELECT visited_on, SUM(amount) AS amount
    FROM Customer
    GROUP BY visited_on
),
growth AS (
    SELECT 
        visited_on,
        SUM(amount) OVER (ORDER BY visited_on RANGE BETWEEN INTERVAL 6 DAY PRECEDING AND CURRENT ROW) AS amount,
        ROUND(SUM(amount) OVER (ORDER BY visited_on RANGE BETWEEN INTERVAL 6 DAY PRECEDING AND CURRENT ROW)/CAST(7 AS float), 2) AS average_amount
    FROM visited_amount
    GROUP BY visited_on
)
SELECT *
FROM growth
WHERE visited_on >= (
        SELECT DATE_ADD(MIN(visited_on), INTERVAL 6 DAY)
        FROM Customer
    )
ORDER BY visited_on
;
""" 

'\nTH visited_amount AS (\n    SELECT visited_on, SUM(amount) AS amount\n    FROM Customer\n    GROUP BY visited_on\n),\ngrowth AS (\n    SELECT \n        visited_on,\n        SUM(amount) OVER (ORDER BY visited_on RANGE BETWEEN INTERVAL 6 DAY PRECEDING AND CURRENT ROW) AS amount,\n        ROUND(SUM(amount) OVER (ORDER BY visited_on RANGE BETWEEN INTERVAL 6 DAY PRECEDING AND CURRENT ROW)/CAST(7 AS float), 2) AS average_amount\n    FROM visited_amount\n    GROUP BY visited_on\n)\nSELECT *\nFROM growth\nWHERE visited_on >= (\n        SELECT DATE_ADD(MIN(visited_on), INTERVAL 6 DAY)\n        FROM Customer\n    )\nORDER BY visited_on\n;\n'